In [2]:
import torch
import torch.nn as nn
import numpy as np

from datasets.openml_datasets import OpenML_Dataset
from datasets.torch_datasets import Torch_Dataset
from utils.weakener import Weakener
from models.model import MLP
from utils.losses import PartialLoss,EMLoss
from utils.trainig_testing import train_and_evaluate,warm_up

In [3]:
Data = OpenML_Dataset('iris', batch_size=64)
Weak = Weakener(Data.num_classes)
#Weak.generate_M(model_class='pll')

In [4]:
print('Methods for Data Class','\n',[method for method in dir(Data) if (method[0] != '_') & callable(getattr(Data, method)) ],'\n'
      'Instances for Data Class','\n',[method for method in dir(Data) if (method[0] != '_') & (method not in [method for method in dir(Data) if (method[0] != '_') & callable(getattr(Data, method)) ]) ],'\n\n'
      'Methods for Weak Class','\n',[method for method in dir(Weak) if (method[0] != '_') & callable(getattr(Weak, method)) ],'\n'
      'Instances for Weak Class','\n',[method for method in dir(Weak) if (method[0] != '_') & (method not in [method for method in dir(Weak) if (method[0] != '_') & callable(getattr(Weak, method)) ]) ])

Methods for Data Class 
 ['get_data', 'get_dataloader', 'include_weak'] 
Instances for Data Class 
 ['batch_size', 'dataset', 'num_classes', 'num_features', 'shuffle', 'splitting_seed', 'test_dataset', 'test_num_samples', 'tr_size', 'train_dataset', 'train_num_samples', 'virtual_labels', 'weak_labels'] 

Methods for Weak Class 
 ['generate_M', 'generate_weak', 'generate_wl_priors', 'label_matrix', 'pll_weights', 'virtual_labels', 'virtual_matrix'] 
Instances for Weak Class 
 ['M', 'c', 'd', 'w', 'z']


In [5]:
Weak.generate_M('pll',pll_p=0.5)
Weak.M[:5,:],Weak.Z[:5,:]

(array([[0.   , 0.375, 0.375],
        [0.375, 0.   , 0.375],
        [0.375, 0.375, 0.   ],
        [0.25 , 0.25 , 0.25 ]]),
 array([[0, 1, 1],
        [1, 0, 1],
        [1, 1, 0],
        [1, 1, 1]]))

In [6]:
train_X,train_y,test_X,test_y =  Data.get_data()
print(train_X.shape)

torch.Size([105, 4])


In [7]:
Weak.generate_weak(train_y)

(tensor([1, 0, 0, 0, 3, 2, 2, 3, 1, 2, 3, 2, 3, 1, 3, 0, 1, 2, 1, 1, 0, 2, 2, 2,
         0, 2, 3, 3, 3, 1, 2, 2, 2, 3, 1, 2, 2, 1, 3, 2, 2, 2, 0, 2, 2, 1, 2, 0,
         1, 2, 3, 3, 0, 0, 0, 2, 0, 3, 0, 2, 1, 3, 3, 1, 2, 0, 1, 3, 0, 1, 3, 2,
         3, 1, 2, 1, 3, 3, 2, 1, 2, 0, 2, 0, 0, 0, 3, 0, 0, 3, 1, 0, 1, 3, 3, 3,
         0, 3, 0, 1, 0, 0, 0, 3, 0], dtype=torch.int32),
 tensor([[1., 0., 1.],
         [0., 1., 1.],
         [0., 1., 1.],
         [0., 1., 1.],
         [1., 1., 1.],
         [1., 1., 0.],
         [1., 1., 0.],
         [1., 1., 1.],
         [1., 0., 1.],
         [1., 1., 0.],
         [1., 1., 1.],
         [1., 1., 0.],
         [1., 1., 1.],
         [1., 0., 1.],
         [1., 1., 1.],
         [0., 1., 1.],
         [1., 0., 1.],
         [1., 1., 0.],
         [1., 0., 1.],
         [1., 0., 1.],
         [0., 1., 1.],
         [1., 1., 0.],
         [1., 1., 0.],
         [1., 1., 0.],
         [0., 1., 1.],
         [1., 1., 0.],
         [1., 1., 1.]

In [8]:
train_y[:5,:],Weak.z[:5],Weak.w[:5,:]

(tensor([[0., 0., 1.],
         [0., 1., 0.],
         [0., 0., 1.],
         [0., 1., 0.],
         [1., 0., 0.]]),
 tensor([1, 0, 0, 0, 3], dtype=torch.int32),
 tensor([[1., 0., 1.],
         [0., 1., 1.],
         [0., 1., 1.],
         [0., 1., 1.],
         [1., 1., 1.]], dtype=torch.float64))

In [9]:
Data.include_weak(Weak.z)

In [10]:
trainloader,testloader = Data.get_dataloader()

In [11]:
print('Methods for MLP Class','\n',[method for method in dir(MLP) if (method[0] != '_') & callable(getattr(MLP, method)) ],'\n'
      'Instances for MLP Class','\n',[method for method in dir(MLP) if (method[0] != '_') & (method not in [method for method in dir(MLP) if (method[0] != '_') & callable(getattr(MLP, method))])])

Methods for MLP Class 
 ['add_module', 'apply', 'bfloat16', 'buffers', 'children', 'cpu', 'cuda', 'double', 'eval', 'extra_repr', 'float', 'forward', 'get_buffer', 'get_extra_state', 'get_parameter', 'get_submodule', 'half', 'ipu', 'load_state_dict', 'modules', 'named_buffers', 'named_children', 'named_modules', 'named_parameters', 'parameters', 'register_backward_hook', 'register_buffer', 'register_forward_hook', 'register_forward_pre_hook', 'register_full_backward_hook', 'register_load_state_dict_post_hook', 'register_module', 'register_parameter', 'requires_grad_', 'set_extra_state', 'share_memory', 'state_dict', 'to', 'to_empty', 'train', 'type', 'xpu', 'zero_grad'] 
Instances for MLP Class 
 ['T_destination', 'dump_patches']


In [12]:
mlp = MLP(Data.num_features,[Data.num_features],Data.num_classes, dropout_p=0.5, bn=True, activation =  'gelu')

In [13]:
mlp

MLP(
  (layers): ModuleList(
    (0): Linear(in_features=4, out_features=4, bias=True)
    (1): Linear(in_features=4, out_features=3, bias=True)
  )
  (batch_norms): ModuleList(
    (0): BatchNorm1d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (dropout): Dropout(p=0.5, inplace=False)
)

In [14]:
logsoftmax = nn.LogSoftmax(dim=1)
inputs, vl, trues, ind = next(iter(trainloader))
v = mlp(inputs)
logp = logsoftmax(v)
p = torch.exp(logp)

In [15]:
Weak.d

4

In [16]:
MM = torch.ones((Weak.d,Weak.c)) / Weak.d
MM

tensor([[0.2500, 0.2500, 0.2500],
        [0.2500, 0.2500, 0.2500],
        [0.2500, 0.2500, 0.2500],
        [0.2500, 0.2500, 0.2500]])

In [17]:
vl = vl.type(torch.LongTensor)

In [23]:
Q = p*MM[vl]

In [24]:
Q

tensor([[0.0164, 0.0452, 0.1884],
        [0.0850, 0.0846, 0.0804],
        [0.0276, 0.0826, 0.1398],
        [0.1047, 0.0817, 0.0636],
        [0.0227, 0.0317, 0.1956],
        [0.0941, 0.0310, 0.1249],
        [0.0529, 0.0030, 0.1941],
        [0.0582, 0.0440, 0.1478],
        [0.1025, 0.0936, 0.0539],
        [0.0207, 0.0399, 0.1894],
        [0.0063, 0.2281, 0.0156],
        [0.0353, 0.1001, 0.1145],
        [0.1228, 0.0637, 0.0636],
        [0.0235, 0.1951, 0.0314],
        [0.1188, 0.0805, 0.0508],
        [0.1043, 0.0844, 0.0613],
        [0.1173, 0.0805, 0.0523],
        [0.0572, 0.0335, 0.1593],
        [0.1047, 0.0817, 0.0636],
        [0.0206, 0.2056, 0.0239],
        [0.0832, 0.0397, 0.1272],
        [0.1133, 0.0755, 0.0612],
        [0.1056, 0.0684, 0.0760],
        [0.0188, 0.0407, 0.1906],
        [0.1155, 0.0687, 0.0658],
        [0.1185, 0.0804, 0.0511],
        [0.0188, 0.0765, 0.1547],
        [0.1153, 0.0438, 0.0910],
        [0.0988, 0.0197, 0.1315],
        [0.115

In [22]:
torch.mean(Q[vl==1,:],0)/torch.sum(torch.mean(Q[vl==1,:],0))

tensor([0.1999, 0.4277, 0.3724], grad_fn=<DivBackward0>)

In [19]:
torch.sum(torch.mean(Q[vl==1,:],0))

tensor(0.2500, grad_fn=<SumBackward0>)

In [20]:
for
Q

SyntaxError: invalid syntax (1527260333.py, line 1)

In [ ]:
optim = torch.optim.Adam(mlp.parameters(),lr=1e-2)
loss = EMLoss(Weak.M)

In [ ]:
mlp, results = train_and_evaluate(mlp,trainloader,testloader,optimizer=optim,loss_fn=loss,num_epochs=50,sound = 1)

In [ ]:
import matplotlib.pyplot as plt
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

# Plot the first subplot (Train Loss)
ax1.plot(results['train_loss'])
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Train Loss')

# Plot the second subplot (Accuracies)
ax2.plot(results['train_acc'], label='Train Accuracy')
ax2.plot(results['test_acc'], label='Test Accuracy')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.set_title('Accuracies')
ax2.legend()

fig.suptitle('EM Learning Results', fontsize=18, fontweight='bold', y=1.05)
# Show the plot
plt.show()

In [ ]:
torch.mean(results['test_acc'][45:])